# Héritage multiple

Python 3 implémente la linearisation C3 pour définir l'ordre d'héritage d'une fonction. Cet algo est différent de celui utilisé en Python 2  (on oublie Python2)

![MRO](https://upload.wikimedia.org/wikipedia/commons/4/47/C3_linearization_example.svg)

 class O
 class A extends O
 class B extends O
 class C extends O
 class D extends O
 class E extends O
 class K1 extends C, B, A
 class K3 extends A, D
 class K2 extends B, D, E
 class Z extends K1, K3, K2


Algo,C3:

Classe elle-même + merge(linéarisation de ses parent+liste des classes dans l'ordre d'écriture sur la classe)
L'algo consiste à prendre le premier élément dans la liste de merge et de le trouver dans le merge pour le "factoriser".

exemple:

K1  hérite de  C, B, A
L(K1) = K1 + merge(L(C),L(B),L(A) + [C,B,A])
L[K1] = K1 + merge([C,0],[B,0],[A,0]+[C,B,A])
le merge prend le premier élément [C] et l'extrait si il est présent sinon on passe au second terme
ici il est présent
L[K1] = [K1,C] + merge([0],[B,0],[A,0],[B,A])
0 n'est pas présent on choisit B
L[K1] = [K1,C,B] + merge([O],[A,0],[A]
On choisit A, reste O
L[K1] = [K1,C,B,A,O] 

Les itération complètes (sur wikipédia)
https://en.wikipedia.org/wiki/C3_linearization

L(O)  := [O]                                                // the linearization of O is trivially the singleton list [O], because O has no parents
 
 L(A)  := [A] + merge(L(O), [O])                             // the linearization of A is A plus the merge of its parents' linearizations with the list of parents...
        = [A] + merge([O], [O])
        = [A, O]                                             // ...which simply prepends A to its single parent's linearization
 
 L(B)  := [B, O]                                             // linearizations of B, C, D and E are computed similar to that of A
 L(C)  := [C, O]
 L(D)  := [D, O]
 L(E)  := [E, O]
 
 L(K1) := [K1] + merge(L(C), L(B), L(A), [C, B, A])          // first, find the linearizations of K1's parents, L(C), L(B), and L(A), and merge them with the parent list [C, B, A]
        = [K1] + merge([C, O], [B, O], [A, O], [C, B, A])    // class C is a good candidate for the first merge step, because it only appears as the head of the first and last lists
        = [K1, C] + merge([O], [B, O], [A, O], [B, A])       // class O is not a good candidate for the next merge step, because it also appears in the tails of list 2 and 3; but class B is a good candidate
        = [K1, C, B] + merge([O], [O], [A, O], [A])          // class A is a good candidate; class O still appears in the tail of list 3
        = [K1, C, B, A] + merge([O], [O], [O])               // finally, class O is a valid candidate, which also exhausts all remaining lists
        = [K1, C, B, A, O]
 
 L(K3) := [K3] + merge(L(A), L(D), [A, D])
        = [K3] + merge([A, O], [D, O], [A, D])               // select A
        = [K3, A] + merge([O], [D, O], [D])                  // fail O, select D
        = [K3, A, D] + merge([O], [O])                       // select O
        = [K3, A, D, O]
 
 L(K2) := [K2] + merge(L(B), L(D), L(E), [B, D, E])
        = [K2] + merge([B, O], [D, O], [E, O], [B, D, E])    // select B
        = [K2, B] + merge([O], [D, O], [E, O], [D, E])       // fail O, select D
        = [K2, B, D] + merge([O], [O], [E, O], [E])          // fail O, select E
        = [K2, B, D, E] + merge([O], [O], [O])               // select O
        = [K2, B, D, E, O]
 
 L(Z)  := [Z] + merge(L(K1), L(K3), L(K2), [K1, K3, K2])
        = [Z] + merge([K1, C, B, A, O], [K3, A, D, O], [K2, B, D, E, O], [K1, K3, K2])    // select K1
        = [Z, K1] + merge([C, B, A, O], [K3, A, D, O], [K2, B, D, E, O], [K3, K2])        // select C
        = [Z, K1, C] + merge([B, A, O], [K3, A, D, O], [K2, B, D, E, O], [K3, K2])        // fail B, select K3
        = [Z, K1, C, K3] + merge([B, A, O], [A, D, O], [K2, B, D, E, O], [K2])            // fail B, fail A, select K2
        = [Z, K1, C, K3, K2] + merge([B, A, O], [A, D, O], [B, D, E, O])                  // select B
        = [Z, K1, C, K3, K2, B] + merge([A, O], [A, D, O], [D, E, O])                     // select A
        = [Z, K1, C, K3, K2, B, A] + merge([O], [D, O], [D, E, O])                        // fail O, select D
        = [Z, K1, C, K3, K2, B, A, D] + merge([O], [O], [E, O])                           // fail O, select E
        = [Z, K1, C, K3, K2, B, A, D, E] + merge([O], [O], [O])                           // select O
        = [Z, K1, C, K3, K2, B, A, D, E, O]                                               // done


L'implémentation en Python et l'appel à la méthode "Method Resolution Order" donne l'arbre d'héritage

In [3]:
class Type(type):
    def __repr__(cls):
        return cls.__name__

> class O(object, metaclass=Type): pass

class A(O): pass

class B(O): pass

class C(O): pass

class D(O): pass

class E(O): pass

class K1(C, B, A): pass

class K3(A, D): pass

class K2(B, D, E): pass

class Z(K1, K3, K2): pass

print(K1.mro())
print(Z.mro())


[K1, C, B, A, O, <class 'object'>]
[Z, K1, C, K3, K2, B, A, D, E, O, <class 'object'>]
